# Evaluate ASIN list for products worth tracking
Specifically, Keepa bestseller results, which are ASINs only, and many Audiobooks mixed in. 

In [1]:
import pandas as pd
import random
import mws
import datetime
import time
from tenacity import *  

import booksnn as bnn

date = datetime.datetime.now().strftime("%Y_%m_%d")

<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject


### Load ASIN list

In [2]:
# Pick the list
all_asins = list(pd.read_table('./asin_lists/asins_keepa_bestsellers_2020_11_30.csv')['column'])

# Set name of output csvs, which probably depend upon ASIN list loaded
moreinfo_csv_name = './asin_lists/asins_keepa_bestsellers_moreinfo_2020_11_30.csv'
result_csv_name = './asin_lists/asins_keepa_bestsellers_interesting_2020_11_30.csv'

/home/lynx/anaconda3/envs/booksnn/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Connect to MWS

In [3]:
products_api = mws.Products(access_key='blahblahblah',
                        secret_key='blahblahblah',
                        account_id='blahblahblah')
marketplace_id = mws.Marketplaces.US.marketplace_id
products_api._use_feature_mwsresponse = True #use new format

### Iterate through asins

In [4]:
i = 0
new_asins_df = pd.DataFrame(columns=[])
for asin in all_asins:
    #print(asin,end=', ')
    time.sleep(0.1) #18000 requests per hour
    try: # will fail if no search results for this asin
        response = products_api.get_matching_product_for_id(marketplace_id, 'ASIN',ids=[asin])
        title = response.parsed.Products.Product.AttributeSets.ItemAttributes.Title

        try: 
            pubdate = response.parsed.Products.Product.AttributeSets.ItemAttributes.PublicationDate
            pubdate = datetime.datetime(int(pubdate[0:4]),int(pubdate[5:7]),int(pubdate[8:10]))
        except: 
            pubdate = datetime.datetime.now().strftime("%Y-%m-%d")

        try: 
            listprice = float(response.parsed.Products.Product.AttributeSets.ItemAttributes.ListPrice.Amount)
        except: 
            listprice = 0

        try: 
            salesrank = response.parsed.Products.Product.SalesRankings.SalesRank[0].Rank
        except: 
            salesrank = 0

        try: 
            num_pages = int(response.parsed.Products.Product.AttributeSets.ItemAttributes.NumberOfPages)
        except: 
            num_pages = 0 #indicates poor listing, or audiobook

        product_info = {'asin': asin,
                        'title': title,
                        'pubdate': pubdate,
                        'listprice': listprice,
                        'salesrank': salesrank,
                        'num_pages': num_pages}
        new_asins_df = new_asins_df.append([product_info],ignore_index=True)

        i = i+1
        if i == 1000: #save results
            print('saving results')
            new_asins_df.to_csv(moreinfo_csv_name,index=False,sep='\t')
            i=0
    except: 
        print('no search results for: '+asin)

saving results
saving results
saving results
saving results
no search results for: 1908476877
saving results
saving results
saving results
saving results
saving results
saving results
no search results for: B08LNBH326
no search results for: B08BF14N6H
no search results for: B086Y5JCHZ
saving results
no search results for: B08L47RWFV
no search results for: B08JF5M43Q
no search results for: B08JDYXPF3
saving results
no search results for: B08HT863TX
saving results
no search results for: 1798735210
no search results for: 1697008062
no search results for: B087HC34WD
saving results
no search results for: B08JF5HND3
no search results for: B08JF5DJFX
saving results
no search results for: B08DBZDFQQ
saving results
saving results
saving results
no search results for: B08B73YTQQ
saving results
no search results for: 153002157X
saving results
saving results
saving results
saving results
saving results
no search results for: 108063133X
saving results
saving results
saving results
saving results
sa

In [5]:
# final save
new_asins_df.to_csv(moreinfo_csv_name,index=False,sep='\t')

In [6]:
i

168

#### Check criteria for interesting product

In [7]:
#check listprice and salesrank
interesting_df = new_asins_df[(new_asins_df['listprice'] > 25) & (new_asins_df['salesrank'] < 2000000)]
#check older than 6mo
interesting_df = interesting_df[interesting_df['pubdate'] < datetime.datetime.now()-datetime.timedelta(days = 180)]
# check nonzero pages
interesting_df = interesting_df[interesting_df['num_pages'] > 0]

#check not in existing list
#asins_in_db = bnn.get_asin_list()
#interesting_df = interesting_df[~interesting_df['asin'].isin(asins_in_db)]


print('Number of ASINs found: ' + str(len(new_asins_df)))
print('Number of them that are interesting: ' + str(len(interesting_df)))

TypeError: '<' not supported between instances of 'str' and 'int'

In [ ]:
new_asins_df[new_asins_df['num_pages']>0]

In [ ]:
new_asins_df